# Dividir una cadena en componentes léxicos

In [ ]:
import shlex

command = "ls -l /home/user"
components = shlex.split(command)
print(components)

# Manejo de comillas

In [ ]:
import shlex

command = 'echo "Hello, shklex!"'
components = shlex.split(command)
print(components)

# Uso de un objeto shlex para un control más fino

In [ ]:
import shlex

command = 'cp "file name with spaces.txt" /destination/'
lexer = shlex.shlex(command, posix=True)
lexer.whitespace_split = True
components = list(lexer)
print(components)

# Análisis léxico de un archivo

In [ ]:
import shlex

commands = """ls -la /home/user
echo "This is a test"
cp "file with spaces.txt" /another/destination
mkdir new_directory
rm -rf old_directory
mv "old file.txt" "new file.txt"
chmod 755 script.sh
chown user:group file.txt
grep "search pattern" file.txt
find /home/user -name "*.txt"
tar -czvf archive.tar.gz /path/to/directory
ping -c 4 example.com
"""

with open("commands.txt", "w") as file:
    file.write(commands)
    
with open('commands.txt', 'r') as file:
    for line in file:
        lexer = shlex.shlex(line, posix=True)
        lexer.whitespace_split = True
        components = list(lexer)
        print(components)


# Uso de comentarios

In [ ]:
import shlex

commands = """ls -la /home/user
# echo "This is a test"
cp "file with spaces.txt" /another/destination
mkdir new_directory
/ this is a comment
rm -rf old_directory
ignore this line
"""

lexer = shlex.shlex(commands, posix=True)
lexer.whitespace_split = True

# omitir lineas o palabras que esten despues de un simbolo especifico, si este entre el simbolo /, elimina toda la palabra
lexer.commenters = [ '/', '#', 'r']
components = list(lexer)
print(components)

# Escape de Cadenas para la Seguridad en Comandos Shell
### `shlex.quote` convierte contenido de variables que contienen caracteres especiales en una forma segura para su uso en comandos de shell. Al envolver la cadena en comillas simples y escapar caracteres como espacios, punto y coma, `&&`, `>`, `|`, comillas y otros, `shlex.quote` evita que estos caracteres sean interpretados de manera no deseada o maliciosa. **Esto asegura que la cadena sea tratada como un texto literal que no se puede ejecutar en una shell o terminal**, para evitar la ejecucion de comandos adicionales o redirigir la salida, protegiendo así contra inyecciones de comandos y errores en la ejecución.

    
### **Lista de Caracteres Especiales dentro del contenidod e variables y su Manejo por `shlex.quote`**

| **Carácter Especial** | **Entrada**              | **Salida con `shlex.quote`**    | **Descripción**                                                   |
|-----------------------|--------------------------|--------------------------------|-------------------------------------------------------------------|
| Espacio               | `file name`              | `'file name'`                  | Los espacios se mantienen intactos pero están rodeados por comillas simples. |
| Punto y coma (`;`)    | `file.txt; rm -rf /`     | `'file.txt\; rm -rf /'`        | El punto y coma se convierte en `\;` para evitar separación de comandos. |
| Doble ampersand (`&&`)| `file.txt && echo "Done"`| `'file.txt \&\& echo "Done"'` | Los `&&` se convierten en `\&\&` para evitar que el shell ejecute comandos condicionalmente. |
| Redirección (`>`)     | `file.txt > output.txt`  | `'file.txt \> output.txt'`    | El `>` se convierte en `\>` para evitar redirección de salida. |
| Barra vertical (`|`) | `command1 | command2`    | `'command1 \| command2'`      | La barra vertical se convierte en `\|` para evitar canalización de salida. |
| Comillas dobles (`"`) | `"text"`                 | `'"text"'`                    | Las comillas dobles se mantienen como parte del texto literal, rodeadas de comillas simples. |
| Comillas simples (`'`) | `'text'`                 | `"'text'"`                    | Las comillas simples se mantienen como parte del texto literal, rodeadas de comillas simples. |
| Signo de dólar (`$`)  | `variable=$value`        | `'variable=\$value'`          | El signo de dólar se convierte en `\$` para evitar expansión de variables. |
| Backticks (`` ` ``)  | `` `command` ``          | ``'\`command\`'``             | Los backticks se convierten en `\`` para evitar la ejecución de comandos. |

In [8]:
import shlex
import subprocess

# Definir comandos potencialmente peligrosos
exportar = 'commands.txt && pip freeze > dependencies.txt'  # Comando para listar y guardar dependencias
eliminar = " && rm -rf /"  # Comando peligroso para borrar todos los archivos

# Construir un comando inseguro
leer_archivo_inseguro = exportar + eliminar
cadena_insegura = shlex.quote(leer_archivo_inseguro)
comando_inseguro = f"ls -l {cadena_insegura}"

# Mostrar el comando antes de aplicar la protección
print(f'Entrada sin escapar: {comando_inseguro}')

# Construir un comando seguro usando shlex.quote
# Escapar solo la parte de la entrada peligrosa
leer_archivo_seguro = 'commands.txt'
cadena_segura = shlex.quote(leer_archivo_seguro)
comando_seguro = f"ls -l {cadena_segura}"

# Mostrar el comando escapado
print(f'Entrada escapada: {comando_seguro}')

# Lista de comandos a ejecutar
comandos_a_ejecutar = [comando_seguro, comando_inseguro]

for comando in comandos_a_ejecutar:
    print()
    try:
        subprocess.run(comando, shell=True, check=True)
        print('Comando ejecutado exitosamente!!!', comando)
    except Exception as e:
        print('No se pudo ejecutar el comando!!!', comando)
        print(f'Error: {e}')
    print()


Entrada sin escapar: ls -l 'commands.txt && pip freeze > dependencies.txt && rm -rf /'
Entrada escapada: ls -l commands.txt

-rw-r--r-- 1 root root 334 Jul 21 02:40 commands.txt
Comando ejecutado exitosamente!!! ls -l commands.txt


No se pudo ejecutar el comando!!! ls -l 'commands.txt && pip freeze > dependencies.txt && rm -rf /'
Error: Command 'ls -l 'commands.txt && pip freeze > dependencies.txt && rm -rf /'' returned non-zero exit status 2.



ls: cannot access 'commands.txt && pip freeze > dependencies.txt && rm -rf /': No such file or directory
